In [ ]:
import pandas as pd

#### preparing menu items for modeling 


In [ ]:
def clean_items_in_list(l, menu_stop_words = [], lemmatize=True):
    # ensures menu item lists are ready for processing: removes stop words, punctuation, digits etc. 
    stemmer = PorterStemmer()
    stop_words = list(ENGLISH_STOP_WORDS) + menu_stop_words
    if isinstance(l, list):
        clean_l = []
        for item in l:
            if isinstance(item, str):
                clean_item = "".join([c.lower() for c in item if c.isascii() or c == " "])
                clean_item = "".join([c for c in clean_item if not c.isdigit()])
                clean_item = " ".join([c.split(".")[-1] for c in clean_item.split() if c not in stop_words])
                clean_item = "".join([c.lower() for c in clean_item if c not in string.punctuation])
                clean_item = " ".join([c.strip() for c in clean_item.split()])
                if lemmatize:
                    clean_item = " ".join([stemmer.stem(c) for c in clean_item.split()])
                if clean_item != "" and clean_item != " ":
                    clean_l.append(clean_item)
        return clean_l
    else:
        return l

In [ ]:
full_combo["Items Processed"] = full_combo["Items"].apply(clean_items_in_list)